In [1]:
#Import Statements

import numpy as np
from numpy.random import default_rng
%matplotlib inline
import matplotlib.pyplot as plt
rng = default_rng()
from datetime import timedelta

import os
import pandas as pd
import time as time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import seaborn as sns
import matplotlib as mpl
import glob
import random
import sys
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable

from os.path import expanduser
home = expanduser("~")
sys.path.append(home+'/repos/ClusNet/code/modules/')

from scipy.ndimage import gaussian_filter
from ClusNet import Cluster
from ClusNet import model as m
from ClusNet import dataset as ds
sys.path.append(home+'/repos/ClusNet/CNN/seg/')
import seg

import glob
import IPython.display as display
import datetime, os
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from IPython.display import clear_output
import matplotlib.gridspec as gridspec
from sklearn.metrics import roc_auc_score

# For more information about autotune:
# https://www.tensorflow.org/guide/data_performance#prefetching
AUTOTUNE = tf.data.experimental.AUTOTUNE
print(f"Tensorflow ver. {tf.__version__}")

Tensorflow ver. 2.4.1


In [2]:
clusglob = glob.glob(home + '/repos/ClusNet/data/eROSITA_no_background/*.fits')

data_length = 200

paths = random.choices(clusglob,k=data_length)
IMG_SIZE = 384
N_CLASSES = 4

In [3]:
modeldir = Cluster.mkdir_model(spath=home+'/repos/ClusNet/models/seg')
x_train,y_train,paths_train,x_test,y_test,paths_test = seg.make_dataset(paths=paths,
                                                                    validation_split=0.90,
                                                                    modeldir=modeldir,
                                                                    shift=False,
                                                                    agn=True,
                                                                    poisson=True,
                                                                    sigma=0.5)



Model directory created --> /Users/erebor/repos/ClusNet/models/seg/AYAWd


TypeError: make_dataset() got an unexpected keyword argument 'paths'

t = tf.keras.utils.to_categorical(y_train[idx], num_classes=3)
print(t.shape)

In [ ]:
idx = np.random.randint(0,x_train.shape[0])
mask = y_train[idx]
image = x_train[idx]

fig, ax = plt.subplots(figsize=(5,5),ncols=2,nrows=2,sharex=True,sharey=True)
idx = np.random.randint(0,image.shape[0])
cmap = mpl.cm.binary
vmin, vmax = np.nanmin(mask), np.nanmax(mask)
norm = mpl.colors.Normalize(vmin=vmin,vmax=vmax)

ax[0][0].imshow(image,
             cmap=cmap,
             interpolation='none',
             norm=norm, aspect = 1)

    
ax[0][1].imshow(mask[:,:,0],
             cmap=mpl.cm.Blues,
             interpolation='none', aspect = 1)
ax[1][0].imshow(mask[:,:,1],
             cmap=mpl.cm.Reds,
             interpolation='none', aspect = 1)
ax[1][1].imshow(mask[:,:,2],
             cmap=mpl.cm.Greens,
             interpolation='none', aspect = 1)

for ax_ in ax.reshape(-1):
    #ax[i].axis('off')
    ax_.set_xticks([])
    ax_.set_yticks([])

plt.tight_layout()
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0, hspace=0)

plt.show()
plt.close()

maybe something other than maxpooling?
running it for longer

In [ ]:
concat = tf.keras.layers.Concatenate(axis=3)
# (batch_size,img_size,img_size,channel)
# (0,1,2,3)
input_shape = (IMG_SIZE,IMG_SIZE,1)

x0 = keras.Input(shape=input_shape)

elayer1 = keras.Sequential()
elayer1.add(layers.Conv2D(64, kernel_size=(3, 3), activation="relu", padding='same'))
elayer1.add(layers.MaxPooling2D((2, 2)))

x1 = elayer1(x0)

elayer2 = keras.Sequential()
elayer2.add(layers.Conv2D(32, kernel_size=(3, 3), activation="relu", padding='same'))
elayer2.add(layers.MaxPooling2D((2, 2)))

x2 = elayer2(x1)

elayer3 = keras.Sequential()
elayer3.add(layers.Conv2D(16, kernel_size=(3, 3), activation="relu", padding='same'))
elayer3.add(layers.MaxPooling2D((2, 2)))

x3 = elayer3(x2)

elayer4 = keras.Sequential()
elayer4.add(layers.Conv2D(8, kernel_size=(3, 3), activation="relu", padding='same'))
elayer4.add(layers.MaxPooling2D((2, 2)))

x4 = elayer4(x3)

dlayer1 = keras.Sequential()
dlayer1.add(layers.Conv2DTranspose(8, kernel_size=(3, 3), activation="relu", padding='same'))
dlayer1.add(layers.UpSampling2D((2, 2)))

x5 = dlayer1(x4)

x6 = concat([x5,x3])

dlayer2 = keras.Sequential()
dlayer2.add(layers.Conv2DTranspose(16, kernel_size=(3, 3), activation="relu", padding='same'))
dlayer2.add(layers.UpSampling2D((2, 2)))

x7 = dlayer2(x6)

x8 = concat([x7, x2])

dlayer3 = keras.Sequential()
dlayer3.add(layers.Conv2DTranspose(32, kernel_size=(3, 3), activation="relu", padding='same'))
dlayer3.add(layers.UpSampling2D((2, 2)))

x9 = dlayer3(x8)

x10 = concat([x9, x1])

dlayer4 = keras.Sequential()
dlayer4.add(layers.Conv2DTranspose(64, kernel_size=(3, 3), activation="relu", padding='same'))
dlayer4.add(layers.UpSampling2D((2, 2)))

x11 = dlayer4(x10)

x12 = concat([x11, x0])

x13 = layers.Conv2DTranspose(3, (1, 1), activation='relu')(x12)

output= x13

In [ ]:
the_U = tf.keras.Model(x0, output)

#loss = tf.keras.metrics.MeanIoU(num_classes=3)
#loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
loss = 'mse'

optimizer=tf.keras.optimizers.Adam(lr=1e-5)


the_U.compile(optimizer=optimizer, 
              loss=loss,
             metrics=['accuracy'])

In [ ]:
the_U.summary()

In [ ]:
epochs = 10
batch_size = 16
validation_split = 0.09
print("\n********LEARNING START********")
start = time.time()
training_start = time.time()
# Train the model on the normalized training set.
history = the_U.fit(x=x_train, 
                    y=y_train, 
                    batch_size=batch_size, 
                    epochs=epochs, 
                    verbose=1, 
                    validation_split=validation_split)

epochs = history.epoch
hist = pd.DataFrame(history.history)

elapsed = time.time() - start
print("\nTIME:",str(timedelta(seconds=elapsed)))
    

In [ ]:
print("\n Evaluate the new model against the test set:")
#print(the_U.evaluate(x=x_test, y=y_test, batch_size=batch_size))

In [ ]:
prediction = the_U(x_test, training=False)

In [ ]:
fig, ax = plt.subplots(figsize=(8,5),nrows=1,ncols=3,sharex=True,sharey=True)
idx = np.random.randint(0,prediction.shape[0])
cmap = mpl.cm.binary
norm = mpl.colors.Normalize(vmin=0, vmax=1)
labels = ['cluster','agn','intersect']
for i in range(3):
    print(np.nanmin(prediction[idx,:,:,i]),np.nanmax(prediction[0,:,:,i]))
    im = ax[i].imshow(prediction[idx,:,:,i],interpolation='none',cmap=cmap,norm=norm)
    ax[i].set_title(labels[i])
#plt.colorbar(im)
cax = fig.add_axes([1, 0.25, 0.01, 0.5])
cbar = mpl.colorbar.ColorbarBase(cax, cmap=cmap, norm = norm)
plt.tight_layout()
plt.subplots_adjust(wspace=0.05)
plt.show()
plt.close()

fig, ax = plt.subplots(figsize=(8,5),nrows=1,ncols=3,sharex=True,sharey=True)
idx = np.random.randint(0,prediction.shape[0])
cmap = mpl.cm.binary
norm = mpl.colors.Normalize(vmin=0, vmax=1)
labels = ['cluster','agn','intersect']
for i in range(3):
    print(np.nanmin(y_test[idx,:,:,i]),np.nanmax(y_test[0,:,:,i]))
    im = ax[i].imshow(y_test[idx,:,:,i],interpolation='none',cmap=cmap,norm=norm)
    ax[i].set_title(labels[i])
#plt.colorbar(im)
cax = fig.add_axes([1, 0.25, 0.01, 0.5])
cbar = mpl.colorbar.ColorbarBase(cax, cmap=cmap, norm = norm)
plt.tight_layout()
plt.subplots_adjust(wspace=0.05)
plt.show()
plt.close()

In [ ]:
fig, ax = plt.subplots(figsize=(7,5),nrows=2,ncols=3,sharex=True,sharey=True)
idx = np.random.randint(0,prediction.shape[0])
cmap = mpl.cm.binary
norm = mpl.colors.Normalize(vmin=0, vmax=1)
labels = ['cluster','agn','intersect']

cmaps = [mpl.cm.Blues,mpl.cm.Reds,mpl.cm.Greens]
for i in range(3):
    
    im = ax[0][i].imshow(y_test[idx,:,:,i],interpolation='none',cmap=cmaps[i],norm=norm)
    im = ax[1][i].imshow(prediction[idx,:,:,i],interpolation='none',cmap=cmaps[i],norm=norm)
    ax[0][i].set_title(labels[i])
    ax[0][i].set_aspect('equal'),ax[1][i].set_aspect('equal')
    ax[0][i].set_xticks([]),ax[1][i].set_xticks([])
    ax[0][i].set_yticks([]),ax[1][i].set_yticks([])

ax[0][0].set_ylabel('Input',fontsize=12)
ax[1][0].set_ylabel('Ouput',fontsize=12)
#cax = fig.add_axes([1, 0.25, 0.01, 0.5])
#cbar = mpl.colorbar.ColorbarBase(cax, cmap=cmap, norm = norm)

plt.subplots_adjust(wspace=-.3,hspace=0)
plt.show()
plt.close()

In [ ]:
def intersection_over_union(prediction, labels):
    pred = tf.argmax(prediction, axis=1)
    labl = tf.constant(labels)
    return iou, conf_mat

In [ ]:

for j in range(10):
    fig, ax = plt.subplots(figsize=(7,7),nrows=3,ncols=3,sharex=True,sharey=True)
    idx = np.random.randint(0,prediction.shape[0])
    cmap = mpl.cm.binary
    norm = mpl.colors.Normalize(vmin=0, vmax=1)
    labels = ['cluster','agn','intersect']

    cmaps = [mpl.cm.Blues,mpl.cm.Reds,mpl.cm.Greens]
    for i in range(3):

        input_ = y_test[idx,:,:,i]
        output_ = prediction[idx,:,:,i]
        contrast = output_-input_
        im = ax[0][i].imshow(input_,interpolation='none',cmap=cmaps[i],norm=norm)
        im = ax[1][i].imshow(output_,interpolation='none',cmap=cmaps[i],norm=norm)
        im = ax[2][i].imshow(contrast,interpolation='none',cmap=cmaps[i],norm=norm)

        ax[0][i].set_title(labels[i])
        for j in range(2):
            ax[j][i].set_aspect('equal')
            ax[j][i].set_xticks([])
            ax[j][i].set_yticks([])

    ax[0][0].set_ylabel('Input',fontsize=12)
    ax[1][0].set_ylabel('Ouput',fontsize=12)
    ax[2][0].set_ylabel('Contrast',fontsize=12)

    #cax = fig.add_axes([1, 0.25, 0.01, 0.5])
    #cbar = mpl.colorbar.ColorbarBase(cax, cmap=cmap, norm = norm)

    plt.subplots_adjust(wspace=-.5,hspace=0)
    plt.show()
    plt.close()
    
    
    m = tf.metrics.MeanIoU(num_classes=4)
    m.update_state(input_, output_)
    mean_iou = m.result().numpy()
    print(mean_iou)

In [ ]:
MEAN_IOUs = []
M500s = []
ROCs =[]
for idx, path in enumerate(paths_test):
    x = Cluster.Cluster(fpath=path)
    
    m = tf.metrics.MeanIoU(num_classes=4)
    true = y_test[idx,:,:,:]
    pred = prediction[idx,:,:,:]
    m.update_state(true,pred)
    mean_iou = m.result().numpy()
    MEAN_IOUs.append(mean_iou)
    
    M500 = np.array(x.M500)
    M500s.append(M500)


In [ ]:
def auroc(true, pred):
    return tf.py_function(roc_auc_score, (true, pred), tf.double)

In [ ]:
print(ROCs)

In [ ]:
plt.figure()
new_x, new_y = zip(*sorted(zip(M500s, MEAN_IOUs)))
plt.plot(new_x,new_y, label='Mean IoU')

plt.xlabel('M500c')
plt.legend()
plt.show()
plt.close()